# Using Naive Bayes algorithm for spam detection

In this assigment, you will predict if a sms message is 'spam' or 'ham' (i.e. not 'spam') using the Bernoulli Naive Bayes *classifier*.

The training data is from the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection.  Please download the zip file before running the code below. 


## Step 1:  Getting, understanding, and cleaning the dataset


###  Importing the dataset

In [1]:
# Import the usual libraries
import numpy as np 
import pandas as pd  # To read in the dataset we will use the Panda's library
df = pd.read_table('SMSSpamCollection', sep = '\t', header=None, names=['label', 'sms_message'])

# Next we observe the first 5 rows of the data to ensure everything was read correctly
df.head() 

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### Data preprocesssing
It would be more convenient if the labels were binary instead of 'ham' and 'spam'.  This way our code can always work with numerical values instead of strings.

In [2]:
df['label']=df.label.map({'spam':1, 'ham':0})
df.head() # Again, lets observe the first 5 rows to make sure everything worked before we continue

,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


### Splitting the dcoument into training set and test set

In [3]:
# This time we will use sklearn's method for seperating the data
from sklearn.model_selection import train_test_split

df_train_msgs, df_test_msgs, df_ytrain, df_ytest = train_test_split(df['sms_message'],df['label'], random_state=0)

#Looking at the train/test split
print("The number of training examples: ", df_train_msgs.shape[0])
print("The number of test exampels: ", df_test_msgs.shape)

print("The first four labels")
print(df_ytrain[0:4])

print("The first four sms messages")
print(df_train_msgs[0:4])


The number of training examples:  4179
The number of test exampels:  (1393,)
The first four labels
872     0
831     1
1273    0
3314    0
Name: label, dtype: int64
The first four sms messages
872     Its going good...no problem..but still need li...
831     U have a secret admirer. REVEAL who thinks U R...
1273                                                Ok...
3314    Huh... Hyde park not in mel ah, opps, got conf...
Name: sms_message, dtype: object


###  Creating the feature vector from the text (feature extraction)

Each message will have its own feature vector.  For each message we will create its feature vector as we discussed in class; we will have a feature for every word in our vocabulary.  The $j$th feature is set to one ($x_j=1$) if the $j$th word from our vocabulary occurs in the message, and set the $j$ feature is set to $0$ otherwise ($x_j=0$).

We will use the sklearn method CountVectorize to create the feature vectors for every messge.

We could have written the code to do this directly by performing the following steps:
* remove capitalization
* remove punctuation 
* tokenize (i.e. split the document into individual words)
* count frequencies of each token 
* remove 'stop words' (these are words that will not help us predict since they occur in most documents, e.g. 'a', 'and', 'the', 'him', 'is' ...

In [4]:
# importing the library
from sklearn.feature_extraction.text import CountVectorizer
# creating an instance of CountVectorizer
# Note there are issues with the way CountVectorizer removes stop words.  To learn more: https://scikit-learn.org/stable/modules/feature_extraction.html#stop-words
vectorizer = CountVectorizer(binary = True, stop_words='english')

# If we wanted to perform Multnomial Naive Bayes, we would include the word counts and use the following code
#vectorizer = CountVectorizer(stop_words='english')

# To see the 'count_vector' object
print(vectorizer)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [5]:
# To see the 'stop words' 
#print(vectorizer.get_stop_words())

In [5]:
# Create the vocabulary for our feature transformation
vectorizer.fit(df_train_msgs)

# Next we create the feature vectors for both the training data and the test data
X_train = vectorizer.transform(df_train_msgs).toarray() # code to turn the training emails into a feature vector
X_test = vectorizer.transform(df_test_msgs).toarray() # code to turn the test email into a feature vector

# Changing the target vectors data type  
y_train = df_ytrain.to_numpy() # Convereting from a Panda series to a numpy array
y_test = df_ytest.to_numpy()

# To observe what the data looks like 
print("The label of the first training example: ", y_train[0])
print("The first training example: ", X_train[0].tolist())# I needed to covernt the datatype to list so all the feature values would be printed

The label of the first training example:  0
The first training example:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Your code goes here

In [7]:
# You should NOT use the features of sklearn library in your code.
#### TO-DO #####

# smoothing value m
m = 0.0

# count for ham and spam message:
ham_count = float(np.sum(y_train == 0))
spam_count = float(np.sum(y_train == 1))

# ==================== (a), (b) ====================
# calculate the estimated value of P(y) for each class y.
p_y0 = ham_count/y_train.size
p_y1 = spam_count/y_train.size

print("The estimated value of P(y) for y = 0: {}".format(p_y0))
print("The estimated value of P(y) for y = 1: {}".format(p_y1))

# Find the estimated phi for the Bernoulli of words "admirer" and "secret"
# ==================== (c) ====================
# word "admirer"
admirer0, admirer1 = 0, 0
index_admirer = vectorizer.vocabulary_['admirer']
for i in range(len(y_train)):
    if y_train[i] == 0:
        admirer0 = admirer0 + X_train[i][index_admirer]
    else:
        admirer1 = admirer1 + X_train[i][index_admirer]
print("phi for admirer for y = 0: {}".format(float(admirer0/ham_count)))
print("phi for admirer for y = 1: {}".format(float(admirer1/spam_count)))

# ==================== (d) ====================
# word "secret"
secret0, secret1 = 0, 0
index_secret = vectorizer.vocabulary_['secret']
for i in range(len(y_train)):
    if y_train[i] == 0:
        secret0 = secret0 + X_train[i][index_secret]
    else:
        secret1 = secret1 + X_train[i][index_secret]
print("phi for secret for y = 0: {}".format(float((secret0+m)/(ham_count+m*2))))
print("phi for secret for y = 1: {}".format(float((secret1+m)/(spam_count+m*2))))


# the estimated phi for the Bernoulli
bern_matrix_y0 = np.array([0 for _ in range(X_train[0].size)]) # class 0 -- ham
bern_matrix_y1 = np.array([0 for _ in range(X_train[0].size)]) # class 1 -- spam
# the number of occurrences of each word
for i, msg in enumerate(X_train):
    if y_train[i] == 0:
        bern_matrix_y0 += msg
    else:
        bern_matrix_y1 += msg

# Note: m is the smoothing value. We can change m's value to smooth or not. 
bern_matrix_y0 = (bern_matrix_y0+m) / (ham_count+m*2)
bern_matrix_y1 = (bern_matrix_y1+m) / (spam_count+m*2)    

print("===============================================")
print("The estimated phi matrix for the Bernoulli when y = 0")
print(bern_matrix_y0.tolist())

print("===============================================")
print("The estimated phi matrix for the Bernoulli when y = 1")
print(bern_matrix_y1.tolist())

# predict the test dataset
y_pre = []
for i, msg in enumerate(X_test):
    p_map0 = np.log(p_y0) # log(p(new email ∣ ham)p(ham))
    p_map1 = np.log(p_y1) # log(p(new email ∣ spam)p(spam))
    
    for j, word in enumerate(msg):
        if word == 1:
            p_map0 += np.log(bern_matrix_y0[j])
            p_map1 += np.log(bern_matrix_y1[j])
        else:
            p_map0 += np.log(1-bern_matrix_y0[j])
            p_map1 += np.log(1-bern_matrix_y1[j])
    y_pre.append(0) if p_map0 > p_map1 else y_pre.append(1)

# The predicted classes for the first 50 test examples
print("The predicted classes for the first 50 test examples: {}".format(y_pre[:50]))

# The predicted classes for the first 5 test examples and last 5 test examples
# ==================== (e) ====================
print("First 5 examples: {}".format(y_pre[:5]))
# ==================== (f) ====================
print("Last 5 examples: {}".format(y_pre[-5:]))

# Total number of test examples classiﬁed correctly
correct_num = 0
for i, j in zip(y_test, y_pre):
    correct_num = correct_num+(i==j)
print("Total number of test examples classiﬁed correctly: {}".format(correct_num))

# Total number of test examples classiﬁed incorrectly.
wrong_num = y_test.size - correct_num
print("Total number of test examples classiﬁed incorrectly: {}".format(wrong_num))

# ==================== (g) ====================
# The percentage error on the test examples
print("The percentage error on the test examples: {}".format(wrong_num/y_test.size))

# zero-R
pre_label = 0 if spam_count < ham_count else 1
zero_pre = [pre_label for _ in range(y_test.size)]

zero_corr = 0 
for i, j in zip(y_test, zero_pre):
    zero_corr = zero_corr + (i==j)
print("The accuracy on the test examples using zero-R: {}".format(zero_corr/y_test.size))
##############

The estimated value of P(y) for y = 0: 0.8655180665230916
The estimated value of P(y) for y = 1: 0.13448193347690834
phi for admirer for y = 0: 0.0
phi for admirer for y = 1: 0.014234875444839857
phi for secret for y = 0: 0.000552944429084877
phi for secret for y = 1: 0.014234875444839857
The estimated phi matrix for the Bernoulli when y = 0
[0.0, 0.0, 0.0002764722145424385, 0.0, 0.0, 0.0, 0.0, 0.0002764722145424385, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

/Users/liyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in log
/Users/liyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: RuntimeWarning: divide by zero encountered in log


The predicted classes for the first 50 test examples: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
First 5 examples: [0, 0, 0, 0, 0]
Last 5 examples: [0, 1, 1, 0, 0]
Total number of test examples classiﬁed correctly: 1325
Total number of test examples classiﬁed incorrectly: 68
The percentage error on the test examples: 0.048815506101938265
The accuracy on the test examples using zero-R: 0.8671931083991385
